In [1]:
import os
import pandas as pd
import sqlite3
from helper_functions import *

In [2]:
files={}
path='data/'
for dir_name,_,filenames in os.walk(path):
    for filename in filenames:
        if filename.endswith('.csv'):
            files[filename[:-4]]=os.path.join(dir_name, filename)
            print(os.path.join(dir_name, filename))

data/customer.csv
data/cust_account.csv
data/cust_churn.csv
data/cust_loc.csv
data/cust_services.csv
data/sample.csv


In [21]:

conn=sqlite3.connect('data/churn.db')
cur=conn.cursor()

for filename in files:
    file=pd.read_csv(files[filename])
    file.to_sql(filename,conn,if_exists='replace',index=False)
    
conn.commit()
table_info(conn,cur)
conn.close()
del(file)

customer
	CustomerID
	Gender
	SeniorCitizen
	Partner
	Dependents

cust_account
	Account_id
	Tenure
	Contract
	PaymentMethod
	PaperlessBilling
	MonthlyCharges
	TotalCharges

cust_churn
	Id
	Churn

cust_loc
	Cust_ID
	State
	Latitude
	Longitude
	ZipCode

cust_services
	Cust_ID
	PhoneService
	MultipleLines
	InternetService
	OnlineSecurity
	OnlineBackup
	DeviceProtection
	TechSupport
	StreamingTV
	StreamingMovies

sample
	Name
	Age



## Creating ER diagrams

https://dbdiagram.io/home

## Data Exploration

In [22]:
conn=sqlite3.connect('data/churn.db')
cur=conn.cursor()

In [6]:
query1="SELECT name FROM sqlite_master WHERE type='table';"
cur.execute(query1).fetchall()

[('customer',),
 ('cust_account',),
 ('cust_churn',),
 ('cust_loc',),
 ('cust_services',),
 ('sample',)]

In [7]:
# Information about customers
query2='SELECT gender,COUNT(*) FROM customer GROUP BY gender;'
cur.execute(query2).fetchall()

[('Female', 3488), ('Male', 3555)]

In [8]:
# Partner
query3='SELECT partner,COUNT(*) FROM customer GROUP BY partner;'
cur.execute(query3).fetchall()

[('No', 3641), ('Yes', 3402)]

In [13]:
# Dependents
query4='SELECT dependents,COUNT(*) FROM customer GROUP BY dependents;'
print(cur.execute(query4).fetchall())


[('No', 5416), ('Yes', 1627)]


In [15]:
# exploring the cust_account table
q5="SELECT avg(tenure) FROM cust_account;"
cur.execute(q5).fetchall()

[(32.37114865824223,)]

In [18]:
# Number of customers who pay higher than average charges
q6="SELECT COUNT(account_id) FROM cust_account WHERE totalcharges>(SELECT AVG(totalcharges) FROM cust_account);"
cur.execute(q6).fetchall()

[(4922,)]

## Joining Tables

In [27]:
cur.execute('''
ALTER TABLE cust_loc
RENAME cust_id to CustomerID
''')

conn.commit()

In [40]:
customer=pd.read_sql('SELECT * FROM customer;',conn)
cust_account=pd.read_sql('SELECT * FROM cust_account;',conn)
cust_churn=pd.read_sql('SELECT * FROM cust_churn;',conn)
cust_loc=pd.read_sql('SELECT * FROM cust_loc;',conn)
cust_services=pd.read_sql('SELECT * FROM cust_services;',conn)

In [41]:
cust_services.rename(columns={'Cust_ID':'CustomerID'},inplace=True)
cust_account.rename(columns={'Account_id':'CustomerID'},inplace=True)
cust_churn.rename(columns={'Id':'CustomerID'},inplace=True)

In [39]:
df_to_join=[customer,cust_account,cust_churn,cust_loc,cust_services]
churn_all=pd.concat(df_to_join,join='inner',axis=1)


,CustomerID,Gender,SeniorCitizen,Partner,Dependents,CustomerID,Tenure,Contract,PaymentMethod,PaperlessBilling,...,CustomerID,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,3668-QPYBK,Male,No,No,No,3668-QPYBK,2,Month-to-month,Mailed check,Yes,...,3668-QPYBK,Yes,No,DSL,Yes,Yes,No,No,No,No
1,9237-HQITU,Female,No,No,Yes,9237-HQITU,2,Month-to-month,Electronic check,Yes,...,9237-HQITU,Yes,No,Fiber optic,No,No,No,No,No,No
2,9305-CDSKC,Female,No,No,Yes,9305-CDSKC,8,Month-to-month,Electronic check,Yes,...,9305-CDSKC,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes
3,7892-POOKP,Female,No,Yes,Yes,7892-POOKP,28,Month-to-month,Electronic check,Yes,...,7892-POOKP,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes
4,0280-XJGEX,Male,No,No,Yes,0280-XJGEX,49,Month-to-month,Bank transfer (automatic),Yes,...,0280-XJGEX,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes


In [44]:
churn_all=churn_all.loc[:,~churn_all.columns.duplicated()]
churn_all.columns

Index(['CustomerID', 'Gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'Tenure', 'Contract', 'PaymentMethod', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'State', 'Latitude',
       'Longitude', 'ZipCode', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies'],
      dtype='object')

In [45]:
# Saving the flattened_table to the database
churn_all.to_sql('churn_all',conn,if_exists='replace',index=False)
conn.commit()

In [46]:
conn.close()